In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import seaborn as sns
import matplotlib.pyplot as plt

from itertools import permutations, combinations_with_replacement, product

from sklearn.linear_model import RidgeClassifier, LogisticRegression, SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split

from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.decomposition import PCA

from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import f1_score, roc_auc_score

import warnings
warnings.filterwarnings('ignore')

In [2]:
df_train = pd.read_csv('train_1.csv')
df_train.shape

(3796, 43)

In [3]:
# df_train.isnull().sum()

In [4]:
X, y = df_train.drop(axis=1, columns=['id', 'label']), df_train.label
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train.shape

(3036, 41)

In [14]:
estimators = [
    ('logReg', LogisticRegression(C=1, class_weight=None, fit_intercept=True, penalty='l1', solver='saga')),    
    ('svc', SVC(C=1, break_ties=False, decision_function_shape='ovr', degree=2, gamma='scale', kernel='poly', shrinking=True)),
    ('ridge', RidgeClassifier(alpha=0.0001, class_weight=None, copy_X=True, fit_intercept=True, normalize=False, positive=False, solver='sag')),
    ('ranFor', RandomForestClassifier(ccp_alpha=0.00043917435221783044, bootstrap=True, class_weight='balanced', criterion='entropy', max_features='log2', max_samples=0.3, min_samples_leaf=1, min_samples_split=4, n_estimators=100, oob_score=False, warm_start=False)),
    ('nnMLP', MLPClassifier(activation='relu', alpha=0.01, hidden_layer_sizes=(30,20,10), learning_rate='adaptive', solver='adam', warm_start=True)),
    ('sgdC', SGDClassifier(alpha=0.001,  class_weight=None,  eta0=1e-06,  fit_intercept=True,  l1_ratio=0,  learning_rate='optimal',  loss='log',  penalty=None,  shuffle=True,  warm_start=True)),
]

weights = list(combinations_with_replacement(range(1, len(estimators)+1), r=len(estimators)))
weights = list(product(range(1, len(estimators)+1), range(1, len(estimators)+1), repeat=3))

weights.append(None)
# print(l)
# weights = [
#     None,
#     [2, 3, 1, 4],    
#     [1, 2, 3, 4]
# ]


pipe = Pipeline([
    ('pca', PCA()),
    ('scale', MinMaxScaler()),           
    ('classifier', VotingClassifier(estimators=estimators, voting='hard')),    
])

## Custom Voting

In [21]:
%%time
pred_df = pd.DataFrame()
pred_df_not = pd.DataFrame()

for estimator in estimators:
    pipe = Pipeline([
#         ('pca', PCA()),
        ('scale', MinMaxScaler()),           
        ('classifier', estimator[1]),    
    ])
    
    pipe.fit(X_train, y_train)
    pred = pipe.predict(X_test)
    pred_df[estimator[0]] = pred
    pred_df_not[estimator[0]] = np.where((pred==0)|(pred==1), pred^1, pred)
pred_df    

CPU times: total: 3.38 s
Wall time: 1.8 s


,logReg,svc,ridge,ranFor,nnMLP,sgdC
0,0,1,0,0,1,1
1,1,1,1,1,1,1
2,1,1,1,1,1,1
3,1,1,1,1,1,1
4,1,1,1,1,1,1
...,...,...,...,...,...,...
755,0,0,0,0,0,0
756,0,0,0,0,1,0
757,1,1,1,1,1,1
758,1,1,1,1,1,1


In [22]:
weights_df = pd.DataFrame(weights, columns=pred_df.columns)
weights_df = weights_df.fillna(1)

weights_df

,logReg,svc,ridge,ranFor,nnMLP,sgdC
0,1.0,1.0,1.0,1.0,1.0,1.0
1,1.0,1.0,1.0,1.0,1.0,2.0
2,1.0,1.0,1.0,1.0,1.0,3.0
3,1.0,1.0,1.0,1.0,1.0,4.0
4,1.0,1.0,1.0,1.0,1.0,5.0
...,...,...,...,...,...,...
46652,6.0,6.0,6.0,6.0,6.0,3.0
46653,6.0,6.0,6.0,6.0,6.0,4.0
46654,6.0,6.0,6.0,6.0,6.0,5.0
46655,6.0,6.0,6.0,6.0,6.0,6.0


In [23]:
%%time

preds_df = pd.DataFrame()

for index, row in weights_df.iterrows():
    sum_1 = np.zeros(pred_df.shape[0])
    sum_0 = np.zeros(pred_df.shape[0])
    col_name = ''
    
    for estimator in estimators:
        est_name = estimator[0]
        col_name+= str(row[est_name]).replace('.0','')
        sum_1+= pred_df[est_name] * row[est_name]
        sum_0+= pred_df_not[est_name] * row[est_name]
    
    pred = np.array((sum_1> sum_0), dtype=int)
    preds_df[col_name] = pred
    print(col_name)
#     print(pred)
preds_df.shape    

111111
111112
111113
111114
111115
111116
111121
111122
111123
111124
111125
111126
111131
111132
111133
111134
111135
111136
111141
111142
111143
111144
111145
111146
111151
111152
111153
111154
111155
111156
111161
111162
111163
111164
111165
111166
111211
111212
111213
111214
111215
111216
111221
111222
111223
111224
111225
111226
111231
111232
111233
111234
111235
111236
111241
111242
111243
111244
111245
111246
111251
111252
111253
111254
111255
111256
111261
111262
111263
111264
111265
111266
111311
111312
111313
111314
111315
111316
111321
111322
111323
111324
111325
111326
111331
111332
111333
111334
111335
111336
111341
111342
111343
111344
111345
111346
111351
111352
111353
111354
111355
111356
111361
111362
111363
111364
111365
111366
111411
111412
111413
111414
111415
111416
111421
111422
111423
111424
111425
111426
111431
111432
111433
111434
111435
111436
111441
111442
111443
111444
111445
111446
111451
111452
111453
111454
111455
111456
111461
111462
111463
111464
111465

116351
116352
116353
116354
116355
116356
116361
116362
116363
116364
116365
116366
116411
116412
116413
116414
116415
116416
116421
116422
116423
116424
116425
116426
116431
116432
116433
116434
116435
116436
116441
116442
116443
116444
116445
116446
116451
116452
116453
116454
116455
116456
116461
116462
116463
116464
116465
116466
116511
116512
116513
116514
116515
116516
116521
116522
116523
116524
116525
116526
116531
116532
116533
116534
116535
116536
116541
116542
116543
116544
116545
116546
116551
116552
116553
116554
116555
116556
116561
116562
116563
116564
116565
116566
116611
116612
116613
116614
116615
116616
116621
116622
116623
116624
116625
116626
116631
116632
116633
116634
116635
116636
116641
116642
116643
116644
116645
116646
116651
116652
116653
116654
116655
116656
116661
116662
116663
116664
116665
116666
121111
121112
121113
121114
121115
121116
121121
121122
121123
121124
121125
121126
121131
121132
121133
121134
121135
121136
121141
121142
121143
121144
121145

125632
125633
125634
125635
125636
125641
125642
125643
125644
125645
125646
125651
125652
125653
125654
125655
125656
125661
125662
125663
125664
125665
125666
126111
126112
126113
126114
126115
126116
126121
126122
126123
126124
126125
126126
126131
126132
126133
126134
126135
126136
126141
126142
126143
126144
126145
126146
126151
126152
126153
126154
126155
126156
126161
126162
126163
126164
126165
126166
126211
126212
126213
126214
126215
126216
126221
126222
126223
126224
126225
126226
126231
126232
126233
126234
126235
126236
126241
126242
126243
126244
126245
126246
126251
126252
126253
126254
126255
126256
126261
126262
126263
126264
126265
126266
126311
126312
126313
126314
126315
126316
126321
126322
126323
126324
126325
126326
126331
126332
126333
126334
126335
126336
126341
126342
126343
126344
126345
126346
126351
126352
126353
126354
126355
126356
126361
126362
126363
126364
126365
126366
126411
126412
126413
126414
126415
126416
126421
126422
126423
126424
126425
126426

135266
135311
135312
135313
135314
135315
135316
135321
135322
135323
135324
135325
135326
135331
135332
135333
135334
135335
135336
135341
135342
135343
135344
135345
135346
135351
135352
135353
135354
135355
135356
135361
135362
135363
135364
135365
135366
135411
135412
135413
135414
135415
135416
135421
135422
135423
135424
135425
135426
135431
135432
135433
135434
135435
135436
135441
135442
135443
135444
135445
135446
135451
135452
135453
135454
135455
135456
135461
135462
135463
135464
135465
135466
135511
135512
135513
135514
135515
135516
135521
135522
135523
135524
135525
135526
135531
135532
135533
135534
135535
135536
135541
135542
135543
135544
135545
135546
135551
135552
135553
135554
135555
135556
135561
135562
135563
135564
135565
135566
135611
135612
135613
135614
135615
135616
135621
135622
135623
135624
135625
135626
135631
135632
135633
135634
135635
135636
135641
135642
135643
135644
135645
135646
135651
135652
135653
135654
135655
135656
135661
135662
135663
135664

144542
144543
144544
144545
144546
144551
144552
144553
144554
144555
144556
144561
144562
144563
144564
144565
144566
144611
144612
144613
144614
144615
144616
144621
144622
144623
144624
144625
144626
144631
144632
144633
144634
144635
144636
144641
144642
144643
144644
144645
144646
144651
144652
144653
144654
144655
144656
144661
144662
144663
144664
144665
144666
145111
145112
145113
145114
145115
145116
145121
145122
145123
145124
145125
145126
145131
145132
145133
145134
145135
145136
145141
145142
145143
145144
145145
145146
145151
145152
145153
145154
145155
145156
145161
145162
145163
145164
145165
145166
145211
145212
145213
145214
145215
145216
145221
145222
145223
145224
145225
145226
145231
145232
145233
145234
145235
145236
145241
145242
145243
145244
145245
145246
145251
145252
145253
145254
145255
145256
145261
145262
145263
145264
145265
145266
145311
145312
145313
145314
145315
145316
145321
145322
145323
145324
145325
145326
145331
145332
145333
145334
145335
145336

...

634512
634513
634514
634515
634516
634521
634522
634523
634524
634525
634526
634531
634532
634533
634534
634535
634536
634541
634542
634543
634544
634545
634546
634551
634552
634553
634554
634555
634556
634561
634562
634563
634564
634565
634566
634611
634612
634613
634614
634615
634616
634621
634622
634623
634624
634625
634626
634631
634632
634633
634634
634635
634636
634641
634642
634643
634644
634645
634646
634651
634652
634653
634654
634655
634656
634661
634662
634663
634664
634665
634666
635111
635112
635113
635114
635115
635116
635121
635122
635123
635124
635125
635126
635131
635132
635133
635134
635135
635136
635141
635142
635143
635144
635145
635146
635151
635152
635153
635154
635155
635156
635161
635162
635163
635164
635165
635166
635211
635212
635213
635214
635215
635216
635221
635222
635223
635224
635225
635226
635231
635232
635233
635234
635235
635236
635241
635242
635243
635244
635245
635246
635251
635252
635253
635254
635255
635256
635261
635262
635263
635264
635265
635266

644151
644152
644153
644154
644155
644156
644161
644162
644163
644164
644165
644166
644211
644212
644213
644214
644215
644216
644221
644222
644223
644224
644225
644226
644231
644232
644233
644234
644235
644236
644241
644242
644243
644244
644245
644246
644251
644252
644253
644254
644255
644256
644261
644262
644263
644264
644265
644266
644311
644312
644313
644314
644315
644316
644321
644322
644323
644324
644325
644326
644331
644332
644333
644334
644335
644336
644341
644342
644343
644344
644345
644346
644351
644352
644353
644354
644355
644356
644361
644362
644363
644364
644365
644366
644411
644412
644413
644414
644415
644416
644421
644422
644423
644424
644425
644426
644431
644432
644433
644434
644435
644436
644441
644442
644443
644444
644445
644446
644451
644452
644453
644454
644455
644456
644461
644462
644463
644464
644465
644466
644511
644512
644513
644514
644515
644516
644521
644522
644523
644524
644525
644526
644531
644532
644533
644534
644535
644536
644541
644542
644543
644544
644545

653425
653426
653431
653432
653433
653434
653435
653436
653441
653442
653443
653444
653445
653446
653451
653452
653453
653454
653455
653456
653461
653462
653463
653464
653465
653466
653511
653512
653513
653514
653515
653516
653521
653522
653523
653524
653525
653526
653531
653532
653533
653534
653535
653536
653541
653542
653543
653544
653545
653546
653551
653552
653553
653554
653555
653556
653561
653562
653563
653564
653565
653566
653611
653612
653613
653614
653615
653616
653621
653622
653623
653624
653625
653626
653631
653632
653633
653634
653635
653636
653641
653642
653643
653644
653645
653646
653651
653652
653653
653654
653655
653656
653661
653662
653663
653664
653665
653666
654111
654112
654113
654114
654115
654116
654121
654122
654123
654124
654125
654126
654131
654132
654133
654134
654135
654136
654141
654142
654143
654144
654145
654146
654151
654152
654153
654154
654155
654156
654161
654162
654163
654164
654165
654166
654211
654212
654213
654214
654215
654216
654221
654222
654223

662656
662661
662662
662663
662664
662665
662666
663111
663112
663113
663114
663115
663116
663121
663122
663123
663124
663125
663126
663131
663132
663133
663134
663135
663136
663141
663142
663143
663144
663145
663146
663151
663152
663153
663154
663155
663156
663161
663162
663163
663164
663165
663166
663211
663212
663213
663214
663215
663216
663221
663222
663223
663224
663225
663226
663231
663232
663233
663234
663235
663236
663241
663242
663243
663244
663245
663246
663251
663252
663253
663254
663255
663256
663261
663262
663263
663264
663265
663266
663311
663312
663313
663314
663315
663316
663321
663322
663323
663324
663325
663326
663331
663332
663333
663334
663335
663336
663341
663342
663343
663344
663345
663346
663351
663352
663353
663354
663355
663356
663361
663362
663363
663364
663365
663366
663411
663412
663413
663414
663415
663416
663421
663422
663423
663424
663425
663426
663431
663432
663433
663434
663435
663436
663441
663442
663443
663444
663445
663446
663451
663452
663453
663454

(760, 46656)

In [24]:
%%time
scores = []
score_list = []
for col in preds_df.columns:
    pred = preds_df[col]
    score = f1_score(y_test, pred, average='micro')
    score_list.append(score)
    scores.append((col, score))

scores[np.argmax(score_list)]

CPU times: total: 56.3 s
Wall time: 59.1 s


('123426', 0.7157894736842105)

In [25]:
%%time
for i in np.argsort(score_list):
    print(scores[i])

('111161', 0.6407894736842106)
('211161', 0.6763157894736842)
('111151', 0.6763157894736842)
('121161', 0.6763157894736842)
('111162', 0.6763157894736842)
('112161', 0.6763157894736842)
('111261', 0.6763157894736842)
('112261', 0.6894736842105263)
('121461', 0.6894736842105263)
('111251', 0.6894736842105263)
('211261', 0.6894736842105263)
('111262', 0.6894736842105263)
('111361', 0.6894736842105263)
('121261', 0.6894736842105263)
('111561', 0.6907894736842105)
('121263', 0.6907894736842105)
('211561', 0.6907894736842105)
('112461', 0.6907894736842105)
('111462', 0.6907894736842105)
('111362', 0.6907894736842105)
('111451', 0.6907894736842105)
('211361', 0.6907894736842105)
('121361', 0.6907894736842105)
('111562', 0.6907894736842105)
('121252', 0.6921052631578948)
('111461', 0.6921052631578948)
('132361', 0.6921052631578948)
('121351', 0.6921052631578948)
('111651', 0.6921052631578948)
('141261', 0.6921052631578948)
('112361', 0.6921052631578948)
('141463', 0.6921052631578948)
('211162

('231352', 0.7013157894736842)
('243251', 0.7013157894736842)
('411262', 0.7013157894736842)
('342561', 0.7013157894736842)
('113151', 0.7013157894736842)
('221263', 0.7013157894736842)
('241552', 0.7013157894736842)
('513161', 0.7013157894736842)
('322362', 0.7013157894736842)
('341361', 0.7013157894736842)
('215563', 0.7013157894736842)
('361561', 0.7013157894736842)
('213352', 0.7013157894736842)
('232161', 0.7013157894736842)
('152652', 0.7013157894736842)
('121265', 0.7013157894736842)
('113342', 0.7013157894736842)
('124351', 0.7013157894736842)
('521563', 0.7013157894736842)
('131253', 0.7013157894736842)
('215664', 0.7013157894736842)
('261461', 0.7013157894736842)
('612142', 0.7013157894736842)
('342662', 0.7013157894736842)
('141251', 0.7013157894736842)
('151261', 0.7013157894736842)
('141252', 0.7013157894736842)
('322161', 0.7013157894736842)
('123162', 0.7013157894736842)
('242662', 0.7013157894736842)
('354161', 0.7013157894736842)
('151262', 0.7013157894736842)
('351651

('221656', 0.7039473684210527)
('431565', 0.7039473684210527)
('431564', 0.7039473684210527)
('123431', 0.7039473684210527)
('523141', 0.7039473684210527)
('221645', 0.7039473684210527)
('123465', 0.7039473684210527)
('313441', 0.7039473684210527)
('411464', 0.7039473684210527)
('123521', 0.7039473684210527)
('621215', 0.7039473684210527)
('324551', 0.7039473684210527)
('431552', 0.7039473684210527)
('123532', 0.7039473684210527)
('313542', 0.7039473684210527)
('123253', 0.7039473684210527)
('123164', 0.7039473684210527)
('123165', 0.7039473684210527)
('621142', 0.7039473684210527)
('411532', 0.7039473684210527)
('222166', 0.7039473684210527)
('222164', 0.7039473684210527)
('123265', 0.7039473684210527)
('222163', 0.7039473684210527)
('442143', 0.7039473684210527)
('251161', 0.7039473684210527)
('152464', 0.7039473684210527)
('222154', 0.7039473684210527)
('152463', 0.7039473684210527)
('431631', 0.7039473684210527)
('431642', 0.7039473684210527)
('431541', 0.7039473684210527)
('313366

('614351', 0.7052631578947368)
('352253', 0.7052631578947368)
('226261', 0.7052631578947368)
('351456', 0.7052631578947368)
('611213', 0.7052631578947368)
('232253', 0.7052631578947368)
('232254', 0.7052631578947368)
('513156', 0.7052631578947368)
('513153', 0.7052631578947368)
('223453', 0.7052631578947368)
('351141', 0.7052631578947368)
('513146', 0.7052631578947368)
('613125', 0.7052631578947368)
('233142', 0.7052631578947368)
('223442', 0.7052631578947368)
('351132', 0.7052631578947368)
('613114', 0.7052631578947368)
('353661', 0.7052631578947368)
('612663', 0.7052631578947368)
('351121', 0.7052631578947368)
('233253', 0.7052631578947368)
('351134', 0.7052631578947368)
('223342', 0.7052631578947368)
('513163', 0.7052631578947368)
('223565', 0.7052631578947368)
('232654', 0.7052631578947368)
('513214', 0.7052631578947368)
('223666', 0.7052631578947368)
('513213', 0.7052631578947368)
('616222', 0.7052631578947368)
('232666', 0.7052631578947368)
('513164', 0.7052631578947368)
('613154

('331133', 0.7052631578947368)
('134153', 0.7052631578947368)
('134152', 0.7052631578947368)
('652641', 0.7052631578947368)
('125261', 0.7052631578947368)
('424165', 0.7052631578947368)
('331154', 0.7052631578947368)
('331132', 0.7052631578947368)
('331152', 0.7052631578947368)
('265165', 0.7052631578947368)
('331144', 0.7052631578947368)
('325252', 0.7052631578947368)
('331141', 0.7052631578947368)
('312431', 0.7052631578947368)
('542661', 0.7052631578947368)
('265164', 0.7052631578947368)
('442265', 0.7052631578947368)
('125163', 0.7052631578947368)
('543265', 0.7052631578947368)
('134631', 0.7052631578947368)
('331441', 0.7052631578947368)
('431251', 0.7052631578947368)
('331365', 0.7052631578947368)
('134565', 0.7052631578947368)
('134564', 0.7052631578947368)
('431254', 0.7052631578947368)
('134554', 0.7052631578947368)
('331356', 0.7052631578947368)
('331354', 0.7052631578947368)
('652354', 0.7052631578947368)
('652355', 0.7052631578947368)
('431246', 0.7052631578947368)
('331344

...

('363432', 0.7105263157894737)
('363354', 0.7105263157894737)
('462625', 0.7105263157894737)
('363411', 0.7105263157894737)
('363412', 0.7105263157894737)
('363414', 0.7105263157894737)
('462614', 0.7105263157894737)
('356253', 0.7105263157894737)
('363415', 0.7105263157894737)
('363416', 0.7105263157894737)
('356252', 0.7105263157894737)
('356246', 0.7105263157894737)
('356245', 0.7105263157894737)
('356244', 0.7105263157894737)
('356242', 0.7105263157894737)
('356236', 0.7105263157894737)
('356235', 0.7105263157894737)
('363422', 0.7105263157894737)
('462513', 0.7105263157894737)
('462311', 0.7105263157894737)
('466136', 0.7105263157894737)
('463422', 0.7105263157894737)
('465121', 0.7105263157894737)
('465122', 0.7105263157894737)
('464141', 0.7105263157894737)
('464136', 0.7105263157894737)
('464135', 0.7105263157894737)
('465126', 0.7105263157894737)
('465131', 0.7105263157894737)
('464142', 0.7105263157894737)
('464126', 0.7105263157894737)
('465136', 0.7105263157894737)
('465144

('426314', 0.7105263157894737)
('426315', 0.7105263157894737)
('426316', 0.7105263157894737)
('426325', 0.7105263157894737)
('426326', 0.7105263157894737)
('442615', 0.7105263157894737)
('442614', 0.7105263157894737)
('442613', 0.7105263157894737)
('426345', 0.7105263157894737)
('426346', 0.7105263157894737)
('426355', 0.7105263157894737)
('426356', 0.7105263157894737)
('426366', 0.7105263157894737)
('426414', 0.7105263157894737)
('265221', 0.7105263157894737)
('426266', 0.7105263157894737)
('426255', 0.7105263157894737)
('265236', 0.7105263157894737)
('426235', 0.7105263157894737)
('443113', 0.7105263157894737)
('265313', 0.7105263157894737)
('265312', 0.7105263157894737)
('542614', 0.7105263157894737)
('443112', 0.7105263157894737)
('542615', 0.7105263157894737)
('542616', 0.7105263157894737)
('442225', 0.7105263157894737)
('265311', 0.7105263157894737)
('265253', 0.7105263157894737)
('265252', 0.7105263157894737)
('265246', 0.7105263157894737)
('265245', 0.7105263157894737)
('426213

('455446', 0.7118421052631578)
('366226', 0.7118421052631578)
('366225', 0.7118421052631578)
('455445', 0.7118421052631578)
('123436', 0.7118421052631578)
('454646', 0.7118421052631578)
('366224', 0.7118421052631578)
('125116', 0.7118421052631578)
('455436', 0.7118421052631578)
('141313', 0.7118421052631578)
('124646', 0.7118421052631578)
('141312', 0.7118421052631578)
('366215', 0.7118421052631578)
('124636', 0.7118421052631578)
('136326', 0.7118421052631578)
('123513', 0.7118421052631578)
('123514', 0.7118421052631578)
('365654', 0.7118421052631578)
('123524', 0.7118421052631578)
('646315', 0.7118421052631578)
('646316', 0.7118421052631578)
('123525', 0.7118421052631578)
('141216', 0.7118421052631578)
('445346', 0.7118421052631578)
('125125', 0.7118421052631578)
('125136', 0.7118421052631578)
('365635', 0.7118421052631578)
('141426', 0.7118421052631578)
('142114', 0.7118421052631578)
('142113', 0.7118421052631578)
('136226', 0.7118421052631578)
('142112', 0.7118421052631578)
('142111

('262412', 0.7118421052631578)
('262213', 0.7118421052631578)
('262214', 0.7118421052631578)
('262215', 0.7118421052631578)
('262216', 0.7118421052631578)
('262326', 0.7118421052631578)
('262325', 0.7118421052631578)
('262226', 0.7118421052631578)
('262322', 0.7118421052631578)
('545536', 0.7118421052631578)
('262321', 0.7118421052631578)
('262316', 0.7118421052631578)
('262311', 0.7118421052631578)
('262312', 0.7118421052631578)
('545526', 0.7118421052631578)
('545515', 0.7118421052631578)
('262313', 0.7118421052631578)
('262111', 0.7118421052631578)
('545215', 0.7118421052631578)
('262414', 0.7118421052631578)
('262416', 0.7118421052631578)
('262625', 0.7118421052631578)
('262624', 0.7118421052631578)
('262613', 0.7118421052631578)
('261316', 0.7118421052631578)
('546216', 0.7118421052631578)
('262524', 0.7118421052631578)
('262523', 0.7118421052631578)
('545315', 0.7118421052631578)
('545316', 0.7118421052631578)
('262516', 0.7118421052631578)
('261414', 0.7118421052631578)
('261415

('136446', 0.7131578947368421)
('112324', 0.7131578947368421)
('112326', 0.7131578947368421)
('324626', 0.7131578947368421)
('112336', 0.7131578947368421)
('434416', 0.7131578947368421)
('452114', 0.7131578947368421)
('226424', 0.7131578947368421)
('324616', 0.7131578947368421)
('342416', 0.7131578947368421)
('342415', 0.7131578947368421)
('154436', 0.7131578947368421)
('324615', 0.7131578947368421)
('156123', 0.7131578947368421)
('242315', 0.7131578947368421)
('362211', 0.7131578947368421)
('112314', 0.7131578947368421)
('112313', 0.7131578947368421)
('544526', 0.7131578947368421)
('342615', 0.7131578947368421)
('155626', 0.7131578947368421)
('344426', 0.7131578947368421)
('263425', 0.7131578947368421)
('263426', 0.7131578947368421)
('352515', 0.7131578947368421)
('155635', 0.7131578947368421)
('342616', 0.7131578947368421)
('155636', 0.7131578947368421)
('155645', 0.7131578947368421)
('362116', 0.7131578947368421)
('256454', 0.7131578947368421)
('436416', 0.7131578947368421)
('344435

In [ ]:
# %%time
# clf = GridSearchCV(pipe, params, cv=5, scoring='f1_micro', verbose=10, refit=True)
# search = clf.fit(X_train, y_train)
# search.best_params_, search.best_score_

In [ ]:
# clf.cv_results_

In [ ]:
# pipe.fit(X_train, y_train)
# pipe.score(X_train, y_train)

y_train_pred = clf.predict(X_train)
ConfusionMatrixDisplay.from_predictions(y_train, y_train_pred)

In [ ]:
print(classification_report(y_train, y_train_pred))

In [ ]:
y_train.value_counts()

In [ ]:
f1_score(y_train, y_train_pred, average='micro')

# Test F1-score

In [ ]:
y_test_pred = clf.predict(X_test)
f1_score(y_test, y_test_pred, average='micro')

# Output

In [ ]:
df_test_final = pd.read_csv('/kaggle/input/students-drop-out-prediction/test.csv')
ids_test_final = df_test_final.id  # row id of final test cases
X_test_final =  df_test_final.drop(axis=1, columns=['id'])
y_test_pred = clf.predict(X_test_final) 
# y_test_pred
submission_df = pd.DataFrame(list(zip(ids_test_final, y_test_pred)), columns =['id', 'label'])
submission_df.to_csv('output.csv', index=False)